TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12
data_path = "D:\\Python\\Wormax_learn2\\preprocessed_data_local_notshuffled\\"
model_name = 'worm_transfer_single_7'
import keras

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                    include_top=False,
                    input_shape=(input_height, input_width, channels))


def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def convolution_feature_extractor(input_height, input_width):
    model = models.Sequential()        
    model.add(conv_base)   
    return model


def define_model():
    model = models.Sequential()
    
    model.add(convolution_feature_extractor(input_height, input_width))   
    model.add(layers.Flatten()) 
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(class_number, activation='softmax'))
    
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
conv_base.trainable = False
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 3, 5, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1966336   
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 16,684,108
Trainable params: 1,969,420
Non-trainable params: 14,714,688
_________________________________________________________________


In [4]:
# little prepocessing
from math import atan2, pi

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, num_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        data = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-1, 12))
        for i in range(len(data)-1):
            data_targets[i] = np.array(to_categorical(get_direction(*data[i+1][1][:2]), num_classes=num_classes))
        data = data[:-1]
        
        # Only X
        data_features = data[:,0]
        
        indexes = np.arange(len(data_features))
        
        if shuffle:
            np.random.shuffle(indexes)
        
        for i in range(0, len(data)-1-batch_size, batch_size):
            samples = data_features.take(indexes[i:i+batch_size],axis=0)
            targets = data_targets.take(indexes[i:i+batch_size],axis=0)
            
            # Weird reshape cuz bug # actually not bug, initial data array is not tensor(idk how to fix)
            samples2 = np.zeros((batch_size, input_height, input_width, channels))
            for j, sample in enumerate(samples):
                samples2[j] = sample
            
            # will not work without this
            samples2 = samples2 / 255
            yield samples2, np.array(targets)

In [6]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 1000:
            break
    classes

In [ ]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=32)
validation_generator = generator(data_path, class_number, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(32, 100, 160, 3)
(32, 12)


### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

In [ ]:
class_weight = {0: 1.0,
                 1: 1.62,
                 2: 2.68,
                 3: 3.36,
                 4: 2.51,
                 5: 1.53,
                 6: 1.0,
                 7: 1.37,
                 8: 2.16,
                 9: 2.61,
                 10: 2.04,
                 11: 1.3}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name + '.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5,                              
        patience=70, 
        min_lr=0.00001
    )
]

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks
                        )

Epoch 1/500
100/100 [==============================] - 25s 252ms/step - loss: 3.6145 - actual_acc: 0.2997 - val_loss: 2.4396 - val_actual_acc: 0.1693
Epoch 2/500
100/100 [==============================] - 24s 242ms/step - loss: 2.8233 - actual_acc: 0.4513 - val_loss: 2.6499 - val_actual_acc: 0.1701
Epoch 3/500
100/100 [==============================] - 28s 275ms/step - loss: 3.3522 - actual_acc: 0.3344 - val_loss: 2.4100 - val_actual_acc: 0.1868
Epoch 4/500
100/100 [==============================] - 24s 244ms/step - loss: 2.5413 - actual_acc: 0.4875 - val_loss: 2.6373 - val_actual_acc: 0.1773
Epoch 5/500
100/100 [==============================] - 25s 247ms/step - loss: 3.3496 - actual_acc: 0.3075 - val_loss: 2.4868 - val_actual_acc: 0.1054
Epoch 6/500
100/100 [==============================] - 25s 248ms/step - loss: 3.1070 - actual_acc: 0.3641 - val_loss: 2.2422 - val_actual_acc: 0.2297
Epoch 7/500
100/100 [==============================] - 26s 261ms/step - loss: 3.4038 - actual_acc: 0

100/100 [==============================] - 28s 280ms/step - loss: 3.2013 - actual_acc: 0.3369 - val_loss: 2.2590 - val_actual_acc: 0.2115
Epoch 56/500
 26/100 [======>.......................] - ETA: 9s - loss: 2.9251 - actual_acc: 0.3810

In [ ]:
from keras import backend as K
K.clear_session()